In [15]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [16]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
#print(mid.describe())
# #bring the time data into a new df:
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
masses = [2,4,12,15,18,28,32,40,44]

#add elapsed time columns
def elapsed_time(df):
    counter = len(df.columns)
    for idx, col in enumerate(df):
        timestamp = pd.to_datetime(df[col], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp - timestamp[0]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[idx]) 
        df.insert(counter, colname, runtime)
        counter += 1
    return df
print(elapsed_time(time))
#print(time)
#print(elapsed_time)
time_sliced = time.iloc[:,9:].fillna(0) #select the time columns
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    for idx, mass in enumerate(masses):
        key = "Timestamp, mass {}".format(mass)
        content = df2.iloc[:, idx].dropna().values
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df.columns

               mass 2.0_time            mass 4.0_time  \
0    2023/05/03 15:36:13.663  2023/05/03 15:36:13.863   
1    2023/05/03 15:36:15.536  2023/05/03 15:36:15.736   
2    2023/05/03 15:36:20.409  2023/05/03 15:36:20.609   
3    2023/05/03 15:36:25.783  2023/05/03 15:36:25.983   
4    2023/05/03 15:36:31.158  2023/05/03 15:36:31.358   
..                       ...                      ...   
690  2023/05/03 16:37:57.989  2023/05/03 16:37:58.189   
691  2023/05/03 16:38:03.363  2023/05/03 16:38:03.563   
692  2023/05/03 16:38:08.738  2023/05/03 16:38:08.938   
693  2023/05/03 16:38:14.113  2023/05/03 16:38:14.313   
694  2023/05/03 16:38:19.488  2023/05/03 16:38:19.688   

              mass 12.0_time           mass 15.0_time  \
0    2023/05/03 15:36:14.063  2023/05/03 15:36:14.264   
1    2023/05/03 15:36:15.936  2023/05/03 15:36:16.136   
2    2023/05/03 15:36:20.809  2023/05/03 15:36:21.209   
3    2023/05/03 15:36:26.183  2023/05/03 15:36:26.384   
4    2023/05/03 15:36:31.558  

Index(['Timestamp, mass 2', 'Timestamp, mass 4', 'Timestamp, mass 12',
       'Timestamp, mass 15', 'Timestamp, mass 18', 'Timestamp, mass 28',
       'Timestamp, mass 32', 'Timestamp, mass 40', 'Timestamp, mass 44'],
      dtype='object')

#### Vaclogger

In [17]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-05-03 15:08:07,0.0,-0.025880,9.980000e-09,2.780000e-09,6.076000e-09,6.498000e-07,-0.000738,0.0,20.202,190.908,-0.001079,0.000008,NaN,NaN
1,NaN,2023-05-03 15:08:16,9.0,-0.025511,9.880000e-09,2.790000e-09,5.910000e-09,6.483000e-07,-0.000709,0.0,20.314,190.908,-0.000769,0.000008,NaN,NaN
2,NaN,2023-05-03 15:08:24,17.0,-0.026274,9.990000e-09,2.780000e-09,5.932000e-09,6.460000e-07,-0.000718,0.0,20.314,190.684,-0.001034,0.000008,NaN,NaN
3,NaN,2023-05-03 15:08:33,26.0,-0.025758,9.900000e-09,2.780000e-09,5.499000e-09,6.429000e-07,-0.000713,0.0,20.314,190.684,-0.000995,0.000008,NaN,NaN
4,NaN,2023-05-03 15:08:41,34.0,-0.025585,9.880000e-09,2.780000e-09,5.953000e-09,6.405000e-07,-0.000716,0.0,20.314,190.684,-0.000946,0.000008,NaN,NaN


#### HiVolta log

In [18]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")

#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]

# hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["I_em"]=abs(hv["I_em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,Unnamed: 19,hv_grid,I_em
0,2023-05-03 15:36:05,03/05/2023,0.000,15:36:05.604,0.0,0.08,0.06,0.06,0.0,0.0,...,-0.0003,0.0072,0.0136,0.0211,0.0015,-0.0127,NaN,NaN,1.630000e-08,4.500000e-09
1,2023-05-03 15:36:06,03/05/2023,1.170,15:36:06.774,0.0,0.08,0.06,0.06,0.0,0.0,...,-0.0003,0.0072,0.0136,0.0211,0.0015,-0.0127,NaN,NaN,1.630000e-08,4.500000e-09
2,2023-05-03 15:36:07,03/05/2023,2.338,15:36:07.942,0.0,0.08,0.06,0.06,0.0,0.0,...,-0.0003,0.0072,0.0136,0.0211,0.0015,-0.0127,NaN,NaN,1.630000e-08,4.500000e-09
3,2023-05-03 15:36:09,03/05/2023,3.546,15:36:09.150,0.0,0.08,0.06,0.06,0.0,0.0,...,-0.0003,0.0072,0.0136,0.0211,0.0015,-0.0127,NaN,NaN,1.630000e-08,4.500000e-09
4,2023-05-03 15:36:10,03/05/2023,4.715,15:36:10.319,0.0,0.08,0.06,0.06,0.0,0.0,...,-0.0003,0.0072,0.0136,0.0211,0.0015,-0.0127,NaN,NaN,1.630000e-08,4.500000e-09


## Data processing

#### Comments

In [19]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'transfer complete' 'no visible esd']


In [20]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
hv_comm.index

[nan 'grid repelling' 'grid transparent' 'increasing bias current'
 'restarting emission' 'grid transparent - no esd'
 'flawed current readings']


Int64Index([ 115,  116,  117,  118, 1097, 1098, 1099, 1100, 1101, 1118, 1119,
            1120, 1121, 1122, 1421, 1422, 1423, 1526, 1527, 1528, 1529, 1530,
            1887, 1888, 1889, 1890, 1891],
           dtype='int64')

#### CernOx temperature R-T conversion

In [21]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Plotting

In [22]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [23]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 Cu ESD EGA in - MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)


plt.savefig(r'./graphs/S1 Cu ESD EGA in_MID.png')
plt.show()


In [24]:
#plotting HiVolta log
plt.figure(figsize=(12,6))

plt.plot(hv["datetime"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["datetime"],hv["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("S1 Cu ESD EGA in - HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.8,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/S1 Cu ESD EGA in_HiVolta.png')
plt.show()


In [25]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 CU ESD sample out - Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/S1 Cu ESD sample out_Pressures.png')
plt.show()

In [26]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 Cu ESD EGA in - Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/S1 Cu ESD EGA in_Temp.png')
plt.show()